In [1]:
import pandas as pd
from sqlalchemy import create_engine
from urllib.parse import urlencode

#this is just to simulate reading from a file
from io import StringIO

In [2]:
# this could also be a .csv file 
# perhaps of barcodes we've scanned ... for example
search_barcodes_file = StringIO("""\
barcode
A000046935391
A000049096878
1048318148010
1039776248012
""")

import_barcodes_df = pd.read_csv(search_barcodes_file)

import_barcodes_df.head(3)

,barcode
0,A000046935391
1,A000049096878
2,1048318148010


In [3]:
# get relevant item data from our db given a location code ... 
location_code = '1cjbg'

sql = """\
select
  location_name.name,
  item.location_code,
  item.barcode,
  item.item_record_num,
  item.bib_record_num,
  item.item_callnumber,
  item.last_checkout_date,
  item.checkout_total,
  item.renewal_total
from
  item
  join "location" on "location".code = item.location_code
  join "location_name" on "location_name".location_id = "location".id
where
  item.location_code = '{}'
""".format(location_code)

db_url = 'https://ilsweb.cincinnatilibrary.org/collection-analysis-testing/current_collection'

# read the results of the above query into a dataframe
location_df = pd.read_csv(
    db_url + '.csv?' + urlencode(query={'sql': sql})
)

print(location_df.head(3))
print('number of items: ', location_df.shape[0])

                                                name  ... renewal_total
0  Main - 1st Floor - Children's Library - Big Books  ...             7
1  Main - 1st Floor - Children's Library - Big Books  ...            29
2  Main - 1st Floor - Children's Library - Big Books  ...             9

[3 rows x 9 columns]
number of items:  56


In [4]:
# this creates a temporary, in-memory database that we can use
engine = create_engine('sqlite://')

# write the dataframes to new tables in the newly created db (`if_exists='replace'` will overwrite any existing table data)
import_barcodes_df.to_sql(
    name='import_barcodes', 
    con=engine, 
    if_exists='replace', 
    index=True # will show the order in which they were scanned / entered
) 
location_df.to_sql(
    name='location_items', 
    con=engine, 
    if_exists='replace'
)

In [5]:
# example of using our in memory db

sql = """\
select
*
from
import_barcodes
"""

# a couple of ways to query

# using the built in method for the sqlalchemy engine
print('sqlalchemy:', engine.execute(sql).fetchall())

print('\n')

# using pandas
print('pandas:', pd.read_sql(sql=sql, con=engine))

sqlalchemy: [(0, 'A000046935391'), (1, 'A000049096878'), (2, '1048318148010'), (3, '1039776248012')]


pandas:    index        barcode
0      0  A000046935391
1      1  A000049096878
2      2  1048318148010
3      3  1039776248012


In [6]:
# these options set the display to show all the columns and rows from resulting dataframes
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

In [7]:
# show item info the items we've scanned (assuming they shared the location code)

sql = """\
select
  import_barcodes.barcode as import_barcode,
  location_items.*
from
  import_barcodes
  join location_items on location_items.barcode = import_barcodes.barcode
order by
  import_barcodes.'index'
"""

pd.read_sql(sql=sql, con=engine)

,import_barcode,index,name,location_code,barcode,item_record_num,bib_record_num,item_callnumber,last_checkout_date,checkout_total,renewal_total
0,A000046935391,12,Main - 1st Floor - Children's Library - Big Books,1cjbg,A000046935391,9164001,3109550,easy,2017-09-27,3,8
1,A000049096878,35,Main - 1st Floor - Children's Library - Big Books,1cjbg,A000049096878,8630045,3115713,easy,2016-01-27,1,1
2,1048318148010,52,Main - 1st Floor - Children's Library - Big Books,1cjbg,1048318148010,4469459,2157377,easy,2016-03-07,16,25
3,1039776248012,53,Main - 1st Floor - Children's Library - Big Books,1cjbg,1039776248012,4661655,1965734,910.91634 ffd818,2021-07-01,6,0


In [8]:
# show the full list of items in the location, with the imported barcodes alongside (null values for missing / unscanned items)

sql = """
select
  import_barcodes.barcode as import_barcode,
  location_items.*
from
  location_items
  left outer join import_barcodes on import_barcodes.barcode = location_items.barcode
order by
  location_items.item_callnumber
"""

output_table = pd.read_sql(sql=sql, con=engine)
output_table

,import_barcode,index,name,location_code,barcode,item_record_num,bib_record_num,item_callnumber,last_checkout_date,checkout_total,renewal_total
0,None,2,Main - 1st Floor - Children's Library - Big Books,1cjbg,A000049592744,8715070,3109552,398.2 ffg149 ru 2015,2019-01-28,6,9
1,None,21,Main - 1st Floor - Children's Library - Big Books,1cjbg,A000048860209,8554619,3109556,423.1 ffw519 2015,2019-09-18,12,28
2,None,45,Main - 1st Floor - Children's Library - Big Books,1cjbg,A000011850351,7278692,2881303,500 fff855 2004,2019-08-14,22,15
3,None,3,Main - 1st Floor - Children's Library - Big Books,1cjbg,A000048860233,8554622,3109555,508.2 ffs659 se 2015,2016-10-14,6,11
4,None,22,Main - 1st Floor - Children's Library - Big Books,1cjbg,A000038420782,7260980,2775748,513.211 ffs275 2010,2015-04-06,7,1
5,None,4,Main - 1st Floor - Children's Library - Big Books,1cjbg,A000036306579,7278685,2881285,551.5 ffm685 2004,2018-10-22,11,27
6,None,5,Main - 1st Floor - Children's Library - Big Books,1cjbg,A000055613467,11331795,3282614,582.16 ffm145 2017,None,0,0
7,None,0,Main - 1st Floor - Children's Library - Big Books,1cjbg,A000036306561,7278686,2881296,595.7 fff855 2004,2019-07-10,18,7
8,1039776248012,53,Main - 1st Floor - Children's Library - Big Books,1cjbg,1039776248012,4661655,1965734,910.91634 ffd818,2021-07-01,6,0
9,None,1,Main - 1st Floor - Children's Library - Big Books,1cjbg,A000024163206,1646247,2658726,easy,2020-08-29,21,29


In [ ]:
# output the table data to an html doc
title = f"CHPL Shelflist—Compare to List: Generated {pd.Timestamp.now(tz='America/New_York').strftime('%Y-%m-%dT%H:%M%z')}"

html_template = f"""\
<!doctype html>
<html lang="en">
  <head>
    <!-- Required meta tags -->
    <meta charset="utf-8">
    <meta name="viewport" content="width=device-width, initial-scale=1, shrink-to-fit=no">

    <!-- Bootstrap CSS -->
    <link rel="stylesheet" href="https://stackpath.bootstrapcdn.com/bootstrap/4.1.3/css/bootstrap.min.css" integrity="sha384-MCw98/SFnGE8fJT3GXwEOngsV7Zt27NXFoaoApmYm81iuXoPkFOJwJ8ERdknLPMO" crossorigin="anonymous">

    <title>{title}</title>
  </head>
  <body>
    <div class="container-fluid">
      <img src='https://raw.githubusercontent.com/plch/brand-and-logos/master/CHPL_Brandmark_Primary.png' alt='CHPL brandmark' width=300px>
      <h1>{title}</h1>

      {output_table.to_html(classes='table table-striped table-sm')}
    </div>
  </body>
</html>
"""

with open("output.html", "w") as output_html:
  output_html.write(html_template)